# Packages

In [ ]:
import GRN
import GRNCreationUtils
from utils import *
from genesGroup import subgraph3N
import pandas as pd
import networkx as nx
import matplotlib as plt

# Functions

In [ ]:
import networkx as nx
import itertools
from collections import Counter
from networkx.algorithms.isomorphism import DiGraphMatcher

def generate_motif_templates():
    """Returns a dictionary of canonical 3-node motif templates."""
    templates = {}

    def add_motif(name, edges):
        G = nx.DiGraph()
        G.add_edges_from(edges)
        templates[name] = G

    # Basic motif types (based on Alon 2002 classification)
    add_motif("Fan-In", [(1, 0), (2, 0)])
    add_motif("Fan-Out", [(0, 1), (0, 2)])
    add_motif("Cascade", [(0, 1), (1, 2)])
    add_motif("FFL", [(0, 1), (1, 2), (0, 2)])
    add_motif("FBL", [(0, 1), (1, 2), (2, 0)])
    add_motif("Bi-Mutual", [(0, 1), (1, 0), (1, 2), (2, 1)])
    add_motif("Clique", [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2), (2, 1)])
    # Add more motifs if needed

    return templates


def classify_subgraph(subG, motif_templates):
    """Matches a 3-node subgraph to a motif type using isomorphism."""
    for name, motifG in motif_templates.items():
        matcher = DiGraphMatcher(subG, motifG)
        if matcher.is_isomorphic():
            return name
    return "Other"


def count_motifs(G):
    """
    Enumerates and classifies all 3-node subgraphs in a directed graph.
    
    Returns:
        motif_counts (Counter): motif type -> count
    """
    motif_templates = generate_motif_templates()
    motif_counts = Counter()
    
    for nodes in itertools.combinations(G.nodes, 3):
        subG = G.subgraph(nodes).copy()
        if nx.is_weakly_connected(subG):  # Optional: only count connected
            motif_type = classify_subgraph(subG, motif_templates)
            motif_counts[motif_type] += 1

    return motif_counts


def randomize_graph(G, n_swaps=1000, seed=None):
    """
    Returns a randomized version of the graph preserving in/out-degree distribution.
    Uses the Maslov-Sneppen edge-rewiring algorithm.
    """
    G_rand = G.copy()
    nx.double_edge_swap(G_rand, n_swaps=n_swaps, max_tries=n_swaps * 10, seed=seed)
    return G_rand

# Load the adjacency matrix as binary
def adj_mx_binary(adj_mx):
    binary_adj_mx = adj_mx.copy()
    binary_adj_mx[binary_adj_mx!=0] = 1.
    return binary_adj_mx

import networkx as nx
import numpy as np
import random as rd

# --- Direction assignment function (star-based) ---
def adjacenteDiMatriceStaredFromGraph(G: nx.Graph,
                                      autoRG: float,
                                      duoRG: float) -> nx.DiGraph:
    """
    Converts an undirected graph into a directed graph
    using a 'star' strategy: the highest-degree node is treated as a hub,
    and edges are oriented outward (with optional bidirectionality and self-loops).
    """
    DiG = nx.DiGraph()
    DiG.add_nodes_from(G)

    # Find the hub node (highest degree)
    degree_dict = dict(G.degree())
    motherNode = max(degree_dict, key=degree_dict.get)
    print(f"Hub node selected for directionality: {motherNode}")

    cache = set()
    distance = nx.single_source_shortest_path_length(G, motherNode)

    for nodeA in distance:
        for nodeB in G[nodeA]:
            edge = (nodeA, nodeB)
            if edge not in cache:
                cache.add(edge)
                cache.add(edge[::-1])
                rdNumber = rd.random()
                if rdNumber < duoRG:
                    DiG.add_edges_from([(nodeA, nodeB), (nodeB, nodeA)])
                else:
                    if distance[nodeA] <= distance[nodeB]:
                        DiG.add_edge(nodeA, nodeB)
                    else:
                        DiG.add_edge(nodeB, nodeA)
        if rd.random() < autoRG:
            DiG.add_edge(nodeA, nodeA)

    return DiG

# --- Connect components probabilistically by degree ---
def connect_components_by_degree(G, hub_bias=1.0):
    """
    Connects disconnected components by adding edges between them.
    Nodes are chosen with probability proportional to (degree ** hub_bias).
    """
    added_edges = 0
    while not nx.is_connected(G):
        components = list(nx.connected_components(G))
        compA, compB = components[0], components[1]

        degrees_A = np.array([G.degree(n) for n in compA])
        degrees_B = np.array([G.degree(n) for n in compB])

        # Amplify hub preference
        degrees_A = (degrees_A + 1e-3) ** hub_bias
        degrees_B = (degrees_B + 1e-3) ** hub_bias

        prob_A = degrees_A / degrees_A.sum()
        prob_B = degrees_B / degrees_B.sum()

        nodeA = np.random.choice(list(compA), p=prob_A)
        nodeB = np.random.choice(list(compB), p=prob_B)

        G.add_edge(nodeA, nodeB)
        added_edges += 1
        print(f"Connected node {nodeA} (deg={G.degree(nodeA)}) with {nodeB} (deg={G.degree(nodeB)})")

    print(f"✅ Graph is now connected (added {added_edges} edges).")
    return G

# --- LFR generation with retry loop ---
def safe_LFR_benchmark(max_retries=10, **kwargs):
    attempt = 0
    while True:
        try:
            attempt += 1
            G = nx.generators.community.LFR_benchmark_graph(**kwargs)
            print(f"✅ Successfully generated LFR graph on attempt {attempt}.")
            return G, attempt
        except nx.ExceededMaxIterations:
            print(f"⚠️ Generation failed on attempt {attempt}, retrying...")
            if attempt >= max_retries:
                raise RuntimeError(f"Exceeded max retries ({max_retries}). LFR generation failed.")

# Full network

In [ ]:
'''
Full network
'''

filepath = 'data/gnw/Ecoli-full.tsv'
adj_mx = adj_mx_gnw_goldstandard(filepath)
plot_grn_degree(adj_mx, plot_network=True)
gnw_properties = graph_properties(adj_mx)
full_graph = nx.from_numpy_array(adj_mx, create_using=nx.DiGraph())

In [ ]:
gnw_motifs = subgraph3N(full_graph)

In [ ]:
motifs_count=get_all_motifs_count(gnw_motifs)

# Plot the motifs count
plot_motifs_count(motifs_count)

# 